**PE Header Features Extractor**

* COFF Header Fields
* Standard Optional Header Fields
* Windows Specific Optional Header Fields
* Direcotroies Entries

*This is just a dumb piece of code, we didn't give any particular attention to the beauty, the optimization or the conventions of python when writing this notebook. If you have any remarks about that please leave a comment we'd love to enhance our coding skills.*

In [1]:
!pip install pefile

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#imports
import sys
import pefile
import pandas as pd
import os.path
from os import listdir
import json
import pandas as pd

In [3]:
#definition of the fields to be extracted
header_features_values = []
#The field Characteristics in the optional header has 
#been devided into
#different characteristics values
#the meaning of those values is as follows:
#Ch1: 
#Ch2:
#Ch3:
#Ch4:
#Ch5:
#Ch6:
#Ch7:
#Ch8:
#Ch9:
#Ch10:
#Ch11:
#Ch12:
#Ch13:
#Ch14:
#Ch15:
#Ch16:
header_features_names = ['NumberOfSections', 'Ch1', 'Ch2', 'Ch3', 'Ch4', 'Ch5',
                         'Ch6', 'Ch7', 'Ch8', 'Ch9', 'Ch10', 'Ch11', 'Ch12',
                         'Ch13', 'Ch14', 'Ch15', 'Ch16',
                        'MajorLinkerVersion',
                        'MinorLinkerVersion','SizeOfCode', 
                        'SizeOfInitializedData',
                        'SizeOfUninitializedData',
                        'MajorOperatingSystemVersion',
                        'MinorOperatingSystemVersion', 'MajorImageVersion', 
                        'MinorImageVersion', 
                        'MajorSubsystemVersion',
                        'MinorSubsystemVersion', 'Unknown', 'Native',
                        'Gui', 'Cui', 'Reserved','Os2Cui', 'Unused', 
                        'PosixCui',
                        'NativeW', 'CeGui', 'EfiApp', 'EfiBsd',
                         'EfiRun', 'EfiRom', 'Xbox', 'Wba',
                        'SizeOfStackReserve',
                        'SizeOfStackCommit', 'SizeOfHeapReserve', 
                        'SizeOfHeapCommit', 
                        'bssVirtualSize', 'bssSizeOfRawData', 
                        'debugVirtualSize', 'debugSizeOfRawData', 
                        'edataVirtualSize', 'edataSizeOfRawData', 
                        'idataVirtualSize', 'idataSizeOfRawData', 
                        'pdataVirtualSize', 'pdataSizeOfRawData', 
                        'rdataVirtualSize', 'rdataSizeOfRawData', 
                        'relocVirtualSize', 'relocSizeOfRawData', 
                        'rsrcVirtualSize', 'rsrcSizeOfRawData', 
                        'tlsVirtualSize', 'tlsSizeOfRawData', 
                        'dataVirtualSize', 'dataSizeOfRawData', 
                        'textVirtualSize', 'textSizeOfRawData', 
                        'ExportTableSize', 
                        'ImportTableSize',
                        'ResourceTableSize',
                        'ExceptionTableSize',
                        'SecurityTableSize', 'BaseRelocSize',
                        'DebugTableSize', 'CopyRightSize',
                        'GlobalPTRSize', 'TlsTableSize',
                        'LoadConfigSize', 'BoundImportSize',
                        'ImportAddressTableSize',
                        'DelayImportDescriptorSize', 'ComDescriptorSize'
                  ]
print(len(header_features_names))

85


In [4]:
def file_header_extractor(fh):
    chn = [fh.NumberOfSections]
    binary_charc = bin(fh.Characteristics)[2:]
    for i in range(16-len(binary_charc)):
        chn.append(0)
    for b in binary_charc:
        chn.append(b)
    return chn

def optional_header_extractor(oh):
    subflg = []
    subsys = oh.Subsystem
    if subsys == 16:
        subsys = 15
    binary_subsys = bin(subsys)[2:]
    for i in range(16-len(binary_subsys)):
        subflg.append(0)
    for b in binary_subsys:
        subflg.append(b)
    try:
        return [oh.MajorLinkerVersion, oh.MinorLinkerVersion, 
                oh.SizeOfCode, oh.SizeOfInitializedData,
                oh.SizeOfUninitializedData,
                oh.MajorOperatingSystemVersion,
                oh.MinorOperatingSystemVersion,
                oh.MajorImageVersion, oh.MinorImageVersion, 
                oh.MajorSubsystemVersion, oh.MinorSubsystemVersion] + subflg + [
                oh.SizeOfStackReserve, oh.SizeOfStackCommit,
                oh.SizeOfHeapReserve, oh.SizeOfHeapCommit
               ]
    except:
         return [oh.MajorLinkerVersion, oh.MinorLinkerVersion, 
                oh.SizeOfCode, oh.SizeOfInitializedData,
                oh.SizeOfUninitializedData,
                oh.MajorOperatingSystemVersion,
                oh.MinorOperatingSystemVersion,
                oh.MajorImageVersion, oh.MinorImageVersion, 
                oh.MajorSubsystemVersion, oh.MinorSubsystemVersion] + subflg + [  
                oh.SizeOfStackReserve, oh.SizeOfStackCommit,
                oh.SizeOfHeapReserve, oh.SizeOfHeapCommit
               ]

def directories_extractor(directories):
    direc_list = []
    #The directories list is not limited to those
    #usually exe files use about 11 directories
    #we included 16 directories but it happened
    #that some exe files used more than 16
    #we just ignored them, PLEASE DON'T DO IT.
    direc = {'IMAGE_DIRECTORY_ENTRY_EXPORT':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_IMPORT':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_RESOURCE':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_EXCEPTION':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_SECURITY':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_BASERELOC':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_DEBUG':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_COPYRIGHT':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_GLOBALPTR':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_TLS':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_LOAD_CONFIG':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_BOUND_IMPORT':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_IAT':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_DELAY_IMPORT':{'address':0, 'value':0},
            'IMAGE_DIRECTORY_ENTRY_COM_DESCRIPTOR':{'address':0, 'value':0},
            }
    for d in directories:
        if d.name in direc.keys():
            direc[d.name]['value']= d.Size
    for key in direc.keys():
        direc_list.append(direc[key]['value'])
    return direc_list

In [5]:
def get_main_code_section(sections, base_of_code):
    addresses = []
    for section in sections: 
        addresses.append(section.VirtualAddress)
    if base_of_code in addresses:   
        return sections[addresses.index(base_of_code)]
    else:
        addresses.append(base_of_code)
        addresses.sort()
        if addresses.index(base_of_code)!= 0:
            return sections[addresses.index(base_of_code)-1]
        else:
            return None

#There could be multiple code sections,
#we take into consideration the main code section only
#and we don't recommend doing that
#to get other code sections please consider using the
#following function
def get_code_sections(sections):
    code_sections=[]
    for section in sections:
        if section.Charateristics == int(0x60000020):
            code_sections.append(section)
    return code_sections

#Since different sections could have any name
#we use flags of every section to locate it
#some sections could be merged into one section
#for example idata, edata, and bss could be all
#included within data section
def get_data_sections(sections, directories):
    data_sections ={'bss':[],'idata':[], 'edata':[], 'rsrc':[], 'pdata':[], 
                    'reloc':[], 'debug':[], 'data':[], 'rdata':[],'tls':[]}
    for section in sections:
        if section.Characteristics == int(0xc0000080):
            data_sections['bss'].append(section)
        elif section.VirtualAddress == directories[1].VirtualAddress:
            data_sections['idata'].append(section)
        elif section.VirtualAddress == directories[0].VirtualAddress:
            data_sections['edata'].append(section)
        elif section.VirtualAddress == directories[2].VirtualAddress:
            data_sections['rsrc'].append(section)
            if section.Name.decode(errors='replace').rstrip('\x00') != '.rsrc':
                print('rsrc Error Maybe'+section.Name.decode(errors='replace').rstrip('\x00'))
        elif section.VirtualAddress == directories[3].VirtualAddress:
            data_sections['pdata'].append(section)
        elif section.VirtualAddress == directories[5].VirtualAddress:
            data_sections['reloc'].append(section)
            if section.Characteristics != int(0x42000040):
                print('reloc Error Maybe')
        elif section.VirtualAddress == directories[6].VirtualAddress:
            data_sections['debug'].append(section)
            if section.Characteristics != int(0x42000040):
                print('debug Error Maybe')
        elif section.VirtualAddress == directories[9].VirtualAddress:
            data_sections['tls'].append(section)
        elif section.Characteristics == int(0xc0000040):
            data_sections['data'].append(section)
        elif section.Characteristics == int(0x40000040):
            data_sections['rdata'].append(section)
    return data_sections

#Exploring sections table!!
def sections_extractor(code_section, data_sections):
    sections_headers = []
    sections = ['bss', 'debug', 'edata', 'idata','pdata', 
                'rdata', 'reloc', 'rsrc','tls','data']
    for section in sections:
        if data_sections[section]:
            sections_headers += [data_sections[section][0].Misc_VirtualSize, 
                                 data_sections[section][0].SizeOfRawData,
                                ]
        else:
            sections_headers += [0,0]
    if code_section:
        sections_headers += [code_section.Misc_VirtualSize,
                             code_section.SizeOfRawData]
    else:
        sections_headers += [0,0]
    return sections_headers
        

In [9]:
#The path to the directory containing exe files samples
path = "../data/samples/"
file = "pink.exe"

rows = []
exe = pefile.PE(path+file)
fh = exe.FILE_HEADER
oh = exe.OPTIONAL_HEADER
sections = exe.sections
base_of_code = oh.BaseOfCode
directories = oh.DATA_DIRECTORY
header_features = file_header_extractor(fh)+optional_header_extractor(oh)+sections_extractor(get_main_code_section(sections, base_of_code),get_data_sections(sections, directories))+ directories_extractor(oh.DATA_DIRECTORY)  
rows.append(header_features)

In [10]:
from IPython.display import display, HTML
data_frame = pd.DataFrame(data=rows, columns=header_features_names)
display(data_frame)

,NumberOfSections,Ch1,Ch2,Ch3,Ch4,Ch5,Ch6,Ch7,Ch8,Ch9,...,BaseRelocSize,DebugTableSize,CopyRightSize,GlobalPTRSize,TlsTableSize,LoadConfigSize,BoundImportSize,ImportAddressTableSize,DelayImportDescriptorSize,ComDescriptorSize
0,3,0,0,0,0,0,0,0,1,0,...,16,0,0,0,0,160,0,0,0,0


In [11]:
data_frame.to_csv()

',NumberOfSections,Ch1,Ch2,Ch3,Ch4,Ch5,Ch6,Ch7,Ch8,Ch9,Ch10,Ch11,Ch12,Ch13,Ch14,Ch15,Ch16,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,MajorOperatingSystemVersion,MinorOperatingSystemVersion,MajorImageVersion,MinorImageVersion,MajorSubsystemVersion,MinorSubsystemVersion,Unknown,Native,Gui,Cui,Reserved,Os2Cui,Unused,PosixCui,NativeW,CeGui,EfiApp,EfiBsd,EfiRun,EfiRom,Xbox,Wba,SizeOfStackReserve,SizeOfStackCommit,SizeOfHeapReserve,SizeOfHeapCommit,bssVirtualSize,bssSizeOfRawData,debugVirtualSize,debugSizeOfRawData,edataVirtualSize,edataSizeOfRawData,idataVirtualSize,idataSizeOfRawData,pdataVirtualSize,pdataSizeOfRawData,rdataVirtualSize,rdataSizeOfRawData,relocVirtualSize,relocSizeOfRawData,rsrcVirtualSize,rsrcSizeOfRawData,tlsVirtualSize,tlsSizeOfRawData,dataVirtualSize,dataSizeOfRawData,textVirtualSize,textSizeOfRawData,ExportTableSize,ImportTableSize,ResourceTableSize,ExceptionTableSize,SecurityTableSize,BaseRelocSize,DebugTableSize,Cop